In [1]:
import numpy as np
from jaqs.data import DataView
from jaqs.research import SignalDigger
import warnings

warnings.filterwarnings("ignore")
dataview_folder = 'stockdata/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

D:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Dataview loaded successfully.


In [2]:
def mask_index_member():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = ~(df_index_member ==0) #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False)
    # 跌停
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

In [15]:
mask = mask_index_member()
can_enter,can_exit = limit_up_down()
price = dv.get_ts('close_adj')
price_bench = dv.data_benchmark

Add formula failed: name [up_limit] exist. Try another name.
Add formula failed: name [down_limit] exist. Try another name.


In [16]:
def add_event():
    dv.add_formula('above', ' Ts_Mean(close_adj, 4) >= Ts_Mean(close_adj, 16)', is_quarterly=False)
    dv.add_formula('delay_above', 'Delay(above, 1)', is_quarterly=False)
    dv.add_formula('sig_cross', 'above&&(!delay_above)', is_quarterly=False)
add_event()

Add formula failed: name [above] exist. Try another name.
Add formula failed: name [delay_above] exist. Try another name.
Add formula failed: name [sig_cross] exist. Try another name.


In [17]:
# 优化事件
from jaqs.research import Optimizer
optimizer = Optimizer(dataview=dv,
                      formula='(Ts_Mean(close_adj, SHORT)>=Ts_Mean(close_adj, LONG))&&(Delay(Ts_Mean(close_adj, SHORT)<Ts_Mean(close_adj, LONG), DELAY))',
                      params={'SHORT':range(2,6,1), 'LONG':range(10,40,5), 'DELAY':range(1,3,1)},
                      name='Cross',
                      price=price,
                      benchmark_price=None,#=None求绝对收益 #=price_bench求相对收益
                      period=5,
                      n_quantiles=5,
                      mask=mask,
                      can_enter=can_enter,
                      can_exit=can_exit,
                      commission=0.0008,#手续费 默认0.0008
                      is_event=True,#是否是事件(0/1因子)
                      is_quarterly=False)#是否是季度因子 默认为False

In [18]:
ret_best = optimizer.enumerate_optimizer(target_type="long_ret",
                                     target="Ann. IR",
                                     in_sample_range=[20160105,20170701],
                                     ascending=False)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 35%
Nan Data Count (should be zero) : 0;  Percentage of effective da

In [19]:
print(ret_best[0]["signal_name"])
print(ret_best[0]["ret"])
print(ret_best[0])

Cross{'SHORT': 5, 'LONG': 35, 'DELAY': 2}
              long_ret  long_short_ret
t-stat        5.102261             NaN
p-value       0.000000             NaN
skewness      1.377845             NaN
kurtosis     12.904474             NaN
Ann. Ret      0.244011             NaN
Ann. Vol      0.379081             NaN
Ann. IR       0.643691             NaN
occurance  3042.000000           362.0
{'ret':               long_ret  long_short_ret
t-stat        5.102261             NaN
p-value       0.000000             NaN
skewness      1.377845             NaN
kurtosis     12.904474             NaN
Ann. Ret      0.244011             NaN
Ann. Vol      0.379081             NaN
Ann. IR       0.643691             NaN
occurance  3042.000000           362.0, 'signal_name': "Cross{'SHORT': 5, 'LONG': 35, 'DELAY': 2}"}
